In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession .\
builder .\
appName("spark") .\
config("spark.sql.warehouse.dir","/user/itv016843/warehouse") .\
enableHiveSupport() .\
master("yarn") .\
getOrCreate()

In [3]:
spark

In [4]:
from pyspark.sql.functions import *

In [5]:
from pyspark.sql import *

# while dealing with data the best practice is to add the current time when the data is cleaning 

In [6]:
! hadoop fs -ls /public/trendytech/lendingclubproject/raw

Found 4 items
drwxr-xr-x   - itv005857 supergroup          0 2023-09-15 14:40 /public/trendytech/lendingclubproject/raw/customers_data_csv
drwxr-xr-x   - itv005857 supergroup          0 2023-09-17 22:57 /public/trendytech/lendingclubproject/raw/loans_data_csv
drwxr-xr-x   - itv005857 supergroup          0 2023-09-18 07:32 /public/trendytech/lendingclubproject/raw/loans_defaulters_csv
drwxr-xr-x   - itv005857 supergroup          0 2023-09-18 07:31 /public/trendytech/lendingclubproject/raw/loans_repayments_csv


In [7]:
df1 = spark.read \
.format("csv") \
.option("header",True) \
.load("/public/trendytech/lendingclubproject/raw/customers_data_csv")

In [8]:
df1.limit(5)

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
b59d80da191f5b573...,null,null,RENT,50000.0,OR,973xx,USA,A,A5,Source Verified,8600.0,Individual,null,null
202d9f56ecb7c3bc9...,police officer,7 years,OWN,85000.0,TX,799xx,USA,A,A5,Source Verified,272384.0,Individual,null,null
e5a140c0922b554b9...,community living ...,6 years,RENT,48000.0,NY,146xx,USA,B,B2,Source Verified,85092.0,Individual,null,null
e12aefc548f750777...,Office,10+ years,OWN,33000.0,CT,067xx,USA,F,F1,Verified,7100.0,Individual,null,null
1b3a50d854fbbf97e...,Special Tooling I...,10+ years,MORTGAGE,81000.0,TX,791xx,USA,E,E5,Verified,190274.0,Individual,null,null


# lets see in how many ways we can clean the null values

# TYPE 1

In [9]:
df1.createOrReplaceTempView("cust_data")

In [10]:
spark.sql("use ajdb1")

""


In [11]:
spark.sql("show tables")

database,tableName,isTemporary
,cust_data,true


In [12]:
df1.limit(3)  # To know which columns to clean i always see table 

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
b59d80da191f5b573...,null,null,RENT,50000.0,OR,973xx,USA,A,A5,Source Verified,8600.0,Individual,null,null
202d9f56ecb7c3bc9...,police officer,7 years,OWN,85000.0,TX,799xx,USA,A,A5,Source Verified,272384.0,Individual,null,null
e5a140c0922b554b9...,community living ...,6 years,RENT,48000.0,NY,146xx,USA,B,B2,Source Verified,85092.0,Individual,null,null


In [13]:
df1.filter("annual_inc is null")

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null


In [14]:
spark.sql("select * from cust_data where annual_inc is null").count()

37

In [15]:
df1.count()

2260701

In [16]:
df1.count() - spark.sql("select * from cust_data where annual_inc is null").count()

2260664

In [17]:
df2 = spark.sql("select * from cust_data where annual_inc is not null")

In [18]:
df2.count()

2260664

In [19]:
df2.limit(5)

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
b59d80da191f5b573...,null,null,RENT,50000.0,OR,973xx,USA,A,A5,Source Verified,8600.0,Individual,null,null
202d9f56ecb7c3bc9...,police officer,7 years,OWN,85000.0,TX,799xx,USA,A,A5,Source Verified,272384.0,Individual,null,null
e5a140c0922b554b9...,community living ...,6 years,RENT,48000.0,NY,146xx,USA,B,B2,Source Verified,85092.0,Individual,null,null
e12aefc548f750777...,Office,10+ years,OWN,33000.0,CT,067xx,USA,F,F1,Verified,7100.0,Individual,null,null
1b3a50d854fbbf97e...,Special Tooling I...,10+ years,MORTGAGE,81000.0,TX,791xx,USA,E,E5,Verified,190274.0,Individual,null,null


# TYPE 2

In [20]:
df1.limit(5)

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
b59d80da191f5b573...,null,null,RENT,50000.0,OR,973xx,USA,A,A5,Source Verified,8600.0,Individual,null,null
202d9f56ecb7c3bc9...,police officer,7 years,OWN,85000.0,TX,799xx,USA,A,A5,Source Verified,272384.0,Individual,null,null
e5a140c0922b554b9...,community living ...,6 years,RENT,48000.0,NY,146xx,USA,B,B2,Source Verified,85092.0,Individual,null,null
e12aefc548f750777...,Office,10+ years,OWN,33000.0,CT,067xx,USA,F,F1,Verified,7100.0,Individual,null,null
1b3a50d854fbbf97e...,Special Tooling I...,10+ years,MORTGAGE,81000.0,TX,791xx,USA,E,E5,Verified,190274.0,Individual,null,null


# ADD time

In [21]:
df3 = df1.withColumn("Time",current_timestamp())

In [22]:
df3.select("Time").limit(5)

Time
2025-07-26 01:49:...
2025-07-26 01:49:...
2025-07-26 01:49:...
2025-07-26 01:49:...
2025-07-26 01:49:...


In [23]:
df3.limit(5)

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint,Time
ca82956941012fe97...,Server/Bartender,3 years,RENT,52000.0,FL,327xx,USA,C,C1,Verified,28821.0,Individual,null,null,2025-07-26 01:49:...
0227159d28405afab...,Transportation co...,10+ years,MORTGAGE,85000.0,MD,211xx,USA,D,D5,Verified,34247.0,Individual,null,null,2025-07-26 01:49:...
eac9f08513f9e824d...,Registered Nurse,3 years,MORTGAGE,75000.0,WI,548xx,USA,C,C5,Not Verified,120306.0,Individual,null,null,2025-07-26 01:49:...
648fd1642abdd66c6...,null,null,RENT,12000.0,FL,330xx,USA,E,E2,Verified,2500.0,Individual,null,null,2025-07-26 01:49:...
c0b13a5a4e9714749...,Office Manager,10+ years,RENT,89000.0,IL,623xx,USA,D,D4,Source Verified,76292.0,Individual,null,null,2025-07-26 01:49:...


In [24]:
df3.filter("annual_inc is null")

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint,Time
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null,2025-07-26 01:49:...
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null,2025-07-26 01:49:...
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null,2025-07-26 01:49:...
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null,2025-07-26 01:49:...
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null,2025-07-26 01:49:...
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null,2025-07-26 01:49:...
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null,2025-07-26 01:49:...
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null,2025-07-26 01:49:...
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null,2025-07-26 01:49:...
e3b0c44298fc1c149...,null,null,null,null,null,null,USA,null,null,null,null,null,null,null,2025-07-26 01:49:...


In [25]:
df3.filter("annual_inc is null").count()

37

In [26]:
df3.count()

2260701

In [27]:
columns = [
    "emp_title", "emp_length", "home_ownership", "annual_inc", "addr_state",
    "zip_code", "country", "grade", "sub_grade", "verification_status",
    "tot_hi_cred_lim", "application_type", "annual_inc_joint", "verification_status_joint"
]

In [28]:
df4 = df3.na.drop(subset = columns)   # use wisely

In [29]:
df4.count()

98112

In [30]:
df4.filter("annual_inc is null")

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint,Time


# continue 

In [31]:
df2.count()

2260664

In [32]:
df2.limit(5)

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
b59d80da191f5b573...,null,null,RENT,50000.0,OR,973xx,USA,A,A5,Source Verified,8600.0,Individual,null,null
202d9f56ecb7c3bc9...,police officer,7 years,OWN,85000.0,TX,799xx,USA,A,A5,Source Verified,272384.0,Individual,null,null
e5a140c0922b554b9...,community living ...,6 years,RENT,48000.0,NY,146xx,USA,B,B2,Source Verified,85092.0,Individual,null,null
e12aefc548f750777...,Office,10+ years,OWN,33000.0,CT,067xx,USA,F,F1,Verified,7100.0,Individual,null,null
1b3a50d854fbbf97e...,Special Tooling I...,10+ years,MORTGAGE,81000.0,TX,791xx,USA,E,E5,Verified,190274.0,Individual,null,null


In [33]:
df5 = df2.withColumn("emp_length",regexp_replace(col("emp_length"),"\D"," "))

In [34]:
df6 = df5.withColumn("emp_length",df5.emp_length.cast("int"))

In [35]:
df6.limit(5)

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
6abfb202a2d279e5b...,Teacher,10,MORTGAGE,70000.0,OH,440xx,USA,A,A2,Source Verified,192719.0,Individual,null,null
62150e21cb59a4f98...,Owner,3,RENT,55000.0,FL,339xx,USA,B,B4,Source Verified,24200.0,Individual,null,null
e3eddc6b60c6a8572...,Director,3,MORTGAGE,275000.0,MO,630xx,USA,C,C3,Verified,616187.0,Individual,null,null
1969aac54bd43471b...,Return processor,1,RENT,31000.0,PA,176xx,USA,C,C4,Not Verified,54451.0,Joint App,75000.0,Not Verified
1ddfbc1602a396578...,Advisor,10,RENT,60000.0,CA,946xx,USA,G,G3,Source Verified,161548.0,Individual,null,null


In [38]:
df6.count()

2260664

In [39]:
df7 = df6.distinct()

In [45]:
df7.count()

2260634

In [46]:
df7.createOrReplaceTempView("cust_data")

In [47]:
spark.sql("select * from cust_data limit 5")

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
1d4e1ef4353b73c00...,Electrician,2,MORTGAGE,50000.0,IL,606xx,USA,C,C3,Not Verified,66064.0,Individual,null,null
cd3fca2113a8df06b...,Filler,10,RENT,50000.0,TX,750xx,USA,E,E3,Verified,65766.0,Individual,null,null
483d0032e0cfb3814...,Pharmacy Technician,4,RENT,32895.0,CA,915xx,USA,B,B5,Not Verified,86191.0,Individual,null,null
d1f96fc86e9db741d...,RN,7,MORTGAGE,95000.0,MI,484xx,USA,B,B1,Not Verified,240839.0,Individual,null,null
f112010a451be99d9...,DSM,4,MORTGAGE,140000.0,CO,801xx,USA,D,D5,Source Verified,360864.0,Individual,null,null


In [48]:
df8 = spark.sql("select floor(avg(emp_length)) as leng from cust_data").collect()

In [51]:
df9 = df8[0][0]

In [53]:
df10 = df7.na.fill(df9,subset=['emp_length'])

In [55]:
df10

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
e6276e5fd0dc1025c...,Director of Finan...,10,MORTGAGE,55000.0,MS,391xx,USA,D,D2,Not Verified,136370.0,Individual,null,null
f97af2fa6c6d36947...,Dock supervisor,10,MORTGAGE,75000.0,UT,840xx,USA,B,B1,Not Verified,291076.0,Individual,null,null
4058b0a68819efa8c...,Sales Manager,9,MORTGAGE,205000.0,FL,320xx,USA,A,A4,Source Verified,543919.0,Individual,null,null
66b0d8f9f34c63822...,Master painter,7,OWN,40000.0,PA,191xx,USA,B,B2,Not Verified,21300.0,Individual,null,null
05f1f2469aa68e446...,Plant manager,4,MORTGAGE,102000.0,GA,304xx,USA,D,D1,Not Verified,344758.0,Individual,null,null
35b43c772817a953d...,Pharmacy Manager,10,MORTGAGE,125000.0,FL,322xx,USA,A,A1,Verified,315302.0,Individual,null,null
670a535867109cc0f...,Social Worker,9,RENT,66000.0,CA,907xx,USA,A,A4,Not Verified,85761.0,Individual,null,null
a39678b787119fca4...,Locomotive Engineer,10,MORTGAGE,90000.0,NJ,077xx,USA,A,A2,Verified,304376.0,Joint App,155000.0,Not Verified
8c2709f002e36c5fb...,Office manager,3,MORTGAGE,30000.0,WV,255xx,USA,B,B3,Verified,149487.0,Individual,null,null
d72e66b19f12bfb0c...,Grocery manager,5,MORTGAGE,33425.6,AL,352xx,USA,B,B2,Verified,191175.0,Individual,null,null


In [56]:
df10.filter("length(addr_state) > 2")

member_id,emp_title,emp_length,home_ownership,annual_inc,addr_state,zip_code,country,grade,sub_grade,verification_status,tot_hi_cred_lim,application_type,annual_inc_joint,verification_status_joint
14cdcfe33d4874cc3...,GSR Holdings Inc,10,RENT,32000.0,and help expand t...,and to be closer ...,USA,C,C2,Source Verified,null,0.0,18252.0,34.0
62f74604b6e23cd77...,COMMUNISPACE CORP,2,RENT,41400.0,debt_consolidation,495.06<br/> Borro...,USA,C,C3,Not Verified,null,0.0,null,Individual
ec384f7b0e9392f89...,Freedom Financial,4,RENT,60000.0,941xx,Paying Off a Loan...,USA,A,A4,Not Verified,0.0,1.0,Individual,null
8a28488a7fb34eb52...,Pyramid Hotel Group,1,RENT,100000.0,000+) currently m...,000 Monthly expe...,USA,D,D5,Verified,null,Jun-2011,Mar-2019,570.0
8d96e25325171af0b...,null,1,RENT,102000.0,and Child Health ...,Caterpillar,USA,D,D4,Not Verified,null,714.0,710.0,null
3ba0b4b60c2058879...,CSC,2,MORTGAGE,64500.0,debt_consolidation,such that I do no...,USA,B,B3,Not Verified,null,0.0,null,Individual
906dc706da2b45c73...,UPS,9,MORTGAGE,50400.0,Get our of debt,debt_consolidation,USA,F,F2,Verified,0.0,null,1.0,null
16ebe35deb55028dc...,FDR Management LLC,2,RENT,52500.0,credit_card,if the $22500 LC ...,USA,D,D2,Verified,null,0.0,null,Individual
7c976e904f66855ad...,SpanSource Techno...,1,MORTGAGE,100000.0,etc. First,or legal arrangem...,USA,D,D1,Verified,null,7393.367160475102,7393.37,1393.37
4131638a0988b03d6...,Barclay Card US,2,MORTGAGE,150000.0,is approximately ...,minus start-up co...,USA,E,E1,Not Verified,null,645.0,0.0,1.0


In [58]:
df11 = df10.withColumn("addr_state",
                      when(length(col("addr_state")) > 2, 'NA').otherwise(col("addr_state"))
                      )

In [60]:
df12 = df11.withColumn(
    "zip_code",
    when(~col("zip_code").rlike("^[0-9]"), "not defined").otherwise(col("zip_code"))
)

In [65]:
df12.groupBy("zip_code like 'n%' ").count()

AnalysisException: cannot resolve '`zip_code like 'n%' `' given input columns: [addr_state, cust_data.annual_inc, cust_data.annual_inc_joint, cust_data.application_type, cust_data.country, emp_length, cust_data.emp_title, cust_data.grade, cust_data.home_ownership, cust_data.member_id, cust_data.sub_grade, cust_data.tot_hi_cred_lim, cust_data.verification_status, cust_data.verification_status_joint, zip_code];
'Aggregate ['zip_code like 'n%' ], ['zip_code like 'n%' , count(1) AS count#3117L]
+- Project [member_id#16, emp_title#17, emp_length#2452, home_ownership#19, annual_inc#20, addr_state#2848, CASE WHEN NOT zip_code#22 RLIKE ^[0-9] THEN not defined ELSE zip_code#22 END AS zip_code#2864, country#23, grade#24, sub_grade#25, verification_status#26, tot_hi_cred_lim#27, application_type#28, annual_inc_joint#29, verification_status_joint#30]
   +- Project [member_id#16, emp_title#17, emp_length#2452, home_ownership#19, annual_inc#20, CASE WHEN (length(addr_state#21) > 2) THEN NA ELSE addr